In [1]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2

In [2]:
model = tf.keras.models.load_model("model_full.h5", compile=False)

Metal device set to: Apple M1 Pro


2022-07-29 18:19:52.023213: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-29 18:19:52.023338: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
df = pd.read_feather('/Volumes/Nathan_disk/dataframe.feather')

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Nathan_disk/dataframe.feather'

In [ ]:
for batch_num, batch_df in tqdm(df.groupby(df.index // BATCH_SIZE)):
    batch = None
    for index in batch_df.index:
        image = cv2.imread(batch_df['filename'].loc[index])
        image_resized = cv2.resize(image, IMG_SIZE, interpolation = cv2.INTER_AREA)
        img_array = tf.keras.preprocessing.image.img_to_array(image_resized)
        single_batch_img_array = tf.expand_dims(img_array, 0)  # Create batch size axis
        if batch is None:
            batch = single_batch_img_array
        else:
            batch = np.concatenate((batch, single_batch_img_array))
    predictions = model.predict_on_batch(batch)
    for pred, i in zip (predictions, batch_df.index):
        df.loc[i, 'label'] = 1 if pred[0] >= 0.5 else 0


  0%|          | 0/3750 [00:00<?, ?it/s]

2022-07-29 16:35:29.411617: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-29 16:35:29.411747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
df.save_feather('labeled.feather')